In [9]:
!cp /volume/fake-news-volume-nfs/tim/news_research/output/user_timeline.json dataset/

In [1]:
import sys
import re
import json
import spacy
import argparse
import pandas as pd
from tqdm import tqdm

from analysis_method import TextRankSummarizer
from util import write_json, read_df, read_stopwords, sentence_to_feature, build_word_dict

## Prepaer the dataset

In [2]:
data_path = "../topic-clustering/outputs/tweets_with_group_ids.csv"
output_path = "outputs/result.json"
stopword_path = "dict/stopwords_en.txt"
text_column = "content"
tweet_id_column = "tweet_id"

In [3]:
nlp = spacy.load("en_core_web_sm")
df = read_df(data_path)
stopwords = read_stopwords(stopword_path)

read data frame from ../topic-clustering/outputs/tweets_with_group_ids.csv
read stopwords from dict/stopwords_en.txt


In [4]:
df['sentences'] = df['content'].apply(lambda v: [s.text for s in nlp(v).sents])

## Extract summary for each group

In [5]:
# convert sentence to features
def build_features(df):
    articles = []
    tweet_ids = []
    for _, row in tqdm(df.iterrows(), desc='filter sentences', total=len(df)):
        sentences = row['sentences']
        if len(sentences) != 0:
            articles.append(sentences)
            tweet_ids.append(row[tweet_id_column])
            
    final_articles, final_tweet_ids = [], []
    for i, article in enumerate(tqdm(articles, desc='convert sentence to feature')):
        sentences = []
        for sent in article:
            tokens = sent.split(' ')
            feature = sentence_to_feature(sent, stopwords)
            if len(feature) > 2:
                sentences.append({'text':sent, 'feature':feature})

        if len(sentences) > 0:
            final_articles.append(sentences)
            final_tweet_ids.append(tweet_ids[i])
    return final_articles, final_tweet_ids

def summary_extraction(final_articles, final_tweet_ids, max_len, topk):
    extractor = TextRankSummarizer(max_len=max_len, allow_selfloop=False)
    result = extractor(final_articles, topk=topk)
    result = [{'sentence':r[0],
               'scores':r[1]
               } for r in result]

    # find tweet that contain summary sentences
    for i, article in enumerate(tqdm(final_articles, desc='find tweet contain summary sentence')):
        for r in result:
            s = r['sentence']
            if any(s in sentence['text'] for sentence in article):
                if r.get('tweet_ids', None) is None:
                    r['tweet_ids'] = []
                r['tweet_ids'].append(final_tweet_ids[i])
    return result

In [6]:
results = {}
topk = 20
max_len = 20
group_size_min_bound = 50

for group_id, subset in df.groupby("group_id"):
    if subset.shape[0] > group_size_min_bound:
        final_articles, final_tweet_ids = build_features(subset)
        results[group_id] = summary_extraction(final_articles, final_tweet_ids, max_len, topk)

find tweet contain summary sentence: 100%|██████████| 56/56 [00:00<00:00, 46557.19it/s]


## Output the result

In [7]:
write_json(results, output_path)

write data to outputs/result.json
